In [ ]:
%pylab notebook

# Pattern Recognition and Machine Learning
# Chapter 2: Probabilitye Distributions

## Probability Distributions: General
* **Density Estimation:** given a finite set $\mathbf{x}_1, \dots, \mathbf{x}_N$ of observations, find the distribution $p(\mathbf{x})$ of $\mathbf{x}$.
  * **Parametric Distributions:** Many usful representations of $p(\mathbf{x})$ depend on some paramters $\theta$, e.g. for a _normal_ distribution $\theta=(\mu, \sigma^2)$
  * **Frequentist's Way:** choose specific parameter $\theta^*$ values by optimizing some criterion (e.g. likelihood)
  * **Bayesian Way:** use a prior distribution over parameters, computer posteriors with Bayes' rule 
    (integrate over all parameters)

> **NOTE:** The same applies to estimating conditional distributions, $p(\mathbf{t} | \mathbf{x})$ and joint distributins $p(\mathbf{t}, \mathbf{x})$. 

* **Conjugate Prior:** leads to a posterior distribution of the same functional form as the prior, which makes life easier. 

## Binary Variable: Frequentist's Way

> **NOTE:** Binary variables are an example of a _discrete_ variable, such as the output of a _classifier_. Understanding how to deal with uncertainty on discrete variables like this will help us to evaluate classification algorithms

Geiven a binary random variable $x \in \{0,1\}$ (e.g. tossing a coin) with 

$$ p(x=1 | \mu) = \mu, \hspace{3em}  p(x=0 | \mu) = 1-\mu.  \hspace{3em} (2.1)$$



$p(x)$ can be described by the _Bernoulli dstribuion:_
$$\begin{align}
\text{Bern}(x|\mu) &= \mu^x (1-\mu)^{1-x}. & (2.2)
\end{align}$$ 

> **NOTE:** Recall $x$ is discrete, so $\text{Bern}(x|\mu)$ is either $\mu$ or $(1-\mu)$, expressing it used an exponential form is mathematically convenient and it will come back to help us later. 

The _maximum likelihood_ estimate for $\mu$ is:
$$\begin{align}
\mu^{ML} &= \frac{m}{N}  \text{  with  }  m=(\text{ #observations of } x = 1) &(2.8)
\end{align}$$ 

> **NOTE:**
> * I derived this on the whiteboard in class, by solving $$\frac{\partial}{\partial \mu} \ln \left\{ \prod_{n=1}^N \text{Bern}(x_n|\mu) \right\}=0,$$ using algebra, logarithmic properties, and the chain rule from calculus. 
> * Maximizing $\ln p(\mathbf{x})$, or mimimizing $-\ln p(\mathbf{x})$, is a useful trick, because it is easier to deal with sums than products when, e.g. calculating the gradient.

This can lead to overfitting, especially for small $N$, e.g. $N=m=3$ yields $\mu^{ML}=1$.

That would mean that a fair coin ($\mu=0.5$) has a $12.5\%$ chance of $m=3$ causing us to beleive that the coin will land head $100\%$ of the time. 

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('gOwLEVQGbrM')

# Binary Variables: Bayesian Way (1), The _Likelihood_

Before going further with a _bayesian_ treatment, we would like to simplify the likelihood function $p(\mathbf{x}|\mu)$. It helps to simplify things by noting that $\mathbf{x}$  has $m$ heads and $N-m$ tails. The probability of seeing $m$ heads is a _binomial_ distribution.

The _binomial distribution_ describes the number $m$ of observations of $x=1$ out of a data set of size $N$:

$$
\begin{align}
\text{Bin}(m|N, \mu) &= {N\choose{m}} \mu^m (1-\mu)^{N-m} & (2.9)\\
{{N}\choose{m}} &= \frac{N!}{(N-m)!m!} & (2.10)
\end{align}
$$

Some explanation:
* For any _specific_ set $\mathbf{x}$ of $m$ heads and $N-m$ tails, the likelihood $p(\mathbf{x}|\mu)=\mu^m (1-\mu)^{(N-m)}$. There are '$N$ _choose_ $m$', or $N\choose{m}$ different inputs $\mathbf{x}$ with $m$ heads, so we add them all together to get $\text{Bin}(m|N,\mu)$. 

In __python__, we can use the [scipy.stats.binom](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html) object to  explore the binomial distribution

In [ ]:
from scipy.stats import binom
print "The odds of 3 heads is: {:0.1%}".format(binom.pmf(3, n=3,p=0.5))

Some explanation:
* The `binom` object provides a number of methods and properties that describe the binomial distribution. 
* The `pmf`, or ___probability mass function___ is the a term for a _discrete_ probability function $p(\cdot)$
* The parameters `n` and `p` correspond to $N$ and $\mu$ in our book. 

> **NOTE:** The design of the `binom` object may seem overwhelming, especially if you are not used to object oriented programming -- it defines a number of methods common to _all_ discrete probability distributions in scipy. Just focus on the part of the interface that we want for now (the probability mass function)

The _binomial_ distribution is a discrete probability function (I mean $m$ takes on discrete values) so let's plot it as a bar chart in python:
> **NOTE:** It would be horribly wrong to use regular plot (lines) to visualize this!

In [ ]:
N = 10
mu = 0.5
figure()
bar(range(N), [binom.pmf(m, n=N, p=mu) for m in range(N)], )
xticks(range(N))
xlabel('$m$')
ylabel('$p(m)$')
title('Bin$(m|N={}, mu={})$'.format(N,mu));

Consider playing with different values of $N$ and $\mu$ above; notice that with $\mu=0.5$, the binomial distribution is a discrete and finite approximation to a normal distribution... is that true for other values of $\mu$?

## Binary Variables: Bayesian Way (2), The _Prior_

For a Bayesian treatment, we need to model the _prior_ distribution $p(\mu)$. It is convenient to model the prior distribution so that it is similar to the posterior (often also the likelihood). This is called a ___conjugate prior___

> **NOTE:** One benefit of this is that your old posterior can become a prior whenever you observe new data, leading to an __incremental__ estimate. 

The _Bernoulli_ distribution has the form $\mu^x(1-\mu)^{1-x}$, so we want a prior distribution that takes on the same structure.  Because it will make future notation easier, let's assume that $$p(\mu|a, b) \propto \mu^{a-1}(1-\mu)^{b-1}$$ 

I was lazy and used the `propto` ($\propto$) symbol above instead of finding the actual probability (which must integrate to one). It may be useful to determine the normalization constant in the formula above, which is problem 2.5 of your textbook. 

If you do that problem (a hint is on the web, and you will need to use the _gamma_ function), you will prove that 
$$ p(\mu | a, b) = \text{Beta}(\mu|a,b) = \frac{\Gamma(a+b)}{\Gamma(a)\Gamma(b)} \mu^{a-1}(1-\mu)^{b-1},$$
which is the _beta_ distribution, and $\Gamma(x)$ is the _gamma_ function,  a continous version of the factorial function, so that $\Gamma(x)=(x-1)!$ when $x$ is a positive integer. 

$$ \Gamma(x) \equiv \int_{0}^{\infty} u^{x-1}e^{-u} du $$

> **NOTE:**  The Gamma function $\Gamma(x)$ is a continous version of the _factorial_ function with $\Gamma(x)=(x-1)!$ for positive integers $x$. 


> **NOTE:** A simple interpretation of hyperparametes $a$ and $b$ is that they are the effective number of observations of $x=1$ and $x=0$ a priori. 

The _mean_ and variance of the _Beta_ distributuion are: 

$$ 
\begin{array}
\mathbb{E}[\mu] &= \frac{a}{a+b}\\
\text{var}[\mu] &= \frac{ab}{(a+b)^2(a+b+1)}\\
\text{mode}[\mu] &= \frac{a-1}{a+b-2}  & \text{(similar to }\frac{m}{N}\text{)}
\end{array} 
$$

> **NOTE:** I, for one, was struck by how similar the __Beta__ and __Binomial__ distributions are, with the notion that $\Gamma$ is similar to a factoral. It seems like the __Beta__ disribution is (approximately) a reparametrized and continous binomial. 

Let's us **python** to explore the _Gamma_ function and _Beta_ distribution.  

Now, let's explore the beta distribution and compare it to the binomial distribution. 
Notice that the seem very similer, if you re-scale the binomial distribution to be in the $[0,1]$ interval. 
Also because the beta distribution is over $\mu$ is is a (continuous) probability densitiy function (PDF) wheras the binomial is discrete, and I plot it using the `step` function to indicate that. 

In [ ]:
from math import gamma,factorial
from scipy.stats import beta

x= linspace(0,1,100)

a = 5
b = 8
figure()
plot(x, [beta.pdf(x_, a, b) for x_ in x])
xlabel(r'$\mu$')
ylabel(r'$p(\mu|a, b)$')

twiny().set_xlabel(r'$m$')
twinx().set_ylabel(r'$p(m|N, \mu)$')

n= float(a+b)
step(arange(a+b+1), [binom.pmf(x_, n, p=a/n) for x_ in arange(a+b+1)], where='mid', color='green');


Some explanation:
* The `gamma` and `factorial` functions are in the `math` module, I import them on line 1. 
* The `beta` distribution is an _object_ in the `scipy.stats` module. It is a type of _continous distribution_.
 * It has a collection of methods such as `beta.pdf` that I can access to plot the probability density function
 * it has some other useful methods with unfortunately short names; For example `beta.rvs` generates [r]andome [v]ariate[s], i.e. random values generated using that distribution. 
 
* The `beta` distribution has two parameters that control the shape, `a` and `b` (they match our text).
* Since I want to play with different values, I define variables (lines 6 and 7) for `a` and `b`
* I want to compare $\text{Beta}$ and $\text{Bin}$, but their horizontal and vertical axes have different scales, so I use `twinx()` ans `twiny()` to inddicate the the axes on the opposite sides of the plot. 
* The `step` function plots a curve as horizontal and vertical lines, I am using it to indicate that $\text{Bin}$ is discrete. 
 * The `where='mid'` argument makes shure the 'step' is halfway betwean each point
 * The `arange` function behaves exactly like `range` except that it returns a numpy array instead of a list. 

Below, I compare the _Gamma_ function to the _factorial_ function

In [ ]:
figure()
x = linspace(1, 6, 100)
plot(x, [gamma(x_) for x_ in x], label='$\Gamma(x)$', color='blue')

scatter(range(1, 7), [factorial(x_-1) for x_ in range(1,7)], color='green', label='$(x-1)!$')
vlines(range(1, 7), 0, [factorial(x_-1) for x_ in range(1,7)], color='green')
legend();

Some explanation:
* I use a _list comprehension_  in order to generate $y$ values for each $x$ in the plot of a blue curve. 
* I also use a trailing underscore to avoid name colision of my temporary variable `x_` and the list `x` of $x$ values. 
* I assign a labels to plots with the '`label=`' parameters. This allows me to display a legend with the `legend()` function. 
* I use `vlines` to drop plumb-lines so you can see that the factorial is evaliated at the integers only.

# Binary Variables: Bayesian Way (3)

Recall that when estimating the probability $p(\mu|...)$ we have 
$$\begin{align}
\text{posterior}&\propto\text{likelihood}\times\text{prior}\\
\underbrace{p(\mu|m, l, a, b)}_{\text{posterior}} &\propto \underbrace{\text{Bin}(m, l|\mu)}_{\text{likelihood}} \underbrace{\text{Beta}(\mu|a,b)}_{\text{prior}}  & \text{(with $l=N-m$)}\\
&\propto \mu^{m+a-1}(1-\mu)^{l+b-1}
\end{align}$$

This leads us to an _**incremental**_ way to estimate update $p(\mu|...)$, which is _extremely_ usful if you want to be able to estimate parameters for large datasets, or if you want to addapt to new observations over time.

> **NOTE:** The _prior_ represents our assumptions; if we are wrong, the prior may hurt the posterior instead of help. _However_, as the number of observaitions $N\rightarrow\infty$ the effect of the prior decreases and we converge to the ML estimate.

# Multinomial Variables: Frequentist's Way

I would like to start with another perspective on the binary _Bernoully_ distribution. 

Let $$\boldsymbol{\mu}
=\left[\begin{array}{c} \mu_1\\ \mu_2\end{array}\right]
=\left[\begin{array}{c} \mu\\ 1-\mu\end{array}\right],$$ where the boldface $\boldsymbol{\mu}$ is used to indicate a vector and the plain $\mu$ is the same parameter used in the previous discussion of _Bernoulli_ distributions. 

> **NOTE:** We will require that that sum $\sum_k \mu_k = 1$, so even though $\boldsymbol{\mu}$ has $2$ values, one of them is determined by that constraint so we only have one degree of freedom.

In addition, we can use an _one-hot_ vector  $\mathbf{x}=[x_1, x_2]$ with $x_1= x$ and $x_2=1-x$. 
Note that **only one element** of $\mathbf{x}$ is ever $1$, and all others are $0$. This is a **one-hot** or **one of k** vector representation of the multinomial variable $x$. 

With this new notation, we can rewrite to the Bernoulli distribution
$$\begin{align}
\text{Bern}(x|\mu) &= \mu^x \times (1-\mu)^{(1-x)} \\
&= \mu_1^{x_1} \times \mu_2^{x_2}\\
&= \prod_{i=1}^2 \mu_i^{x_i}
\end{align}
$$

More generally, a random variable with $K$ _mutually exclusive_ states can be represented as a $K$ dimensional vector $\mathbf{x}$ with $x_k=1$ and all other $x_{i\neq k}=0$. 

We can _generalize_ the Bernoulli distribution to handle $K$ states by setting
$$p(\mathbf{x}|\boldsymbol{\mu})=\prod_{k=1}^K \mu_k^{x_k}$$
with the constraint $\sum_k \mu_k=1$. 

Recall that for _binary_ values $x$ the likelihood of observing a dataset $\mathcal{D}$ of $N$ independant variables could be characterized by the _binomial_ distribution, which was paramtrized by the number of times we observed $x=1$. For $K$-ary variables we have the _multinomial_ distribution
$$\text{Mult}(m_1, m_2, \dots,m_K|\boldsymbol{\mu},N) 
= \left(\begin{array}{c}N\\m_1 m_2 \dots m_K\end{array}\right) \prod_{k=1}^K\mu_k^{m_k}$$

where we define
$$\left(\begin{array}{c}N\\m_1 m_2 \dots m_K\end{array}\right)\equiv \frac{N!}{m_1!m_2!\dots m_K!}$$

Here again we have the constraint that $$\sum_{k=1}^K m_k = N$$ because each $m_k$ is the number of times we observed $x=k$ in $N$ trials

For binary varaibles, we found the $\text{Beta}$ distribution as a conjugate prior that was very similar to the _binomial_ likelihood distribution, using the $\Gamma$-function instead of factorials. For $K$-ary variable we use the _multinomial_ distribution for the likelihood and the _Dirichlet distribution_ as a conjugate prior:
$$ 
\text{Dir}(\boldsymbol{\mu}|\boldsymbol{\alpha}) 
= \frac{\Gamma(\alpha_0)}{\Gamma(\alpha_1)\dots\Gamma(\alpha_K)} \prod_{k=1}^K \mu_k^{\alpha_k-1}
$$

Here we set $\alpha_0=\sum_{k=1}^K \alpha_k$, so $\text{Dir}$ is very similar to $\text{Mult}$ in the same manner that $\text{Beta}$ is similar to $\text{Bin}$. 

### Plots

In [ ]:
from scipy.stats import dirichlet

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

figure()
# alphas = (6,2,2)
# alphas = (3,7,5)
# alphas = (6,2,6)
alphas = (2,3,4)
# alphas = (.9,.9, .9)
# alphas = (1,1,1)

ax = gca(projection='3d')

# Generate mu1 and mu2 coordinate on a grid
mu1, mu2 = mgrid[0:1:100j, 0:1:100j]

# We cannot handle mu==0, and we require mu1 + mu2 + mu3 < 1
eps = 1e-6
mu1 = mu1.clip(eps, 1-2*eps)
mu2 = mu2.clip(eps, 1.-mu1-eps)
mu3 = 1.0 - mu1 - mu2

# Evaluate the PDF
z = zeros_like(mu1)
z.flat = [dirichlet.pdf(mu, alphas) for mu in zip(mu1.flat, mu2.flat, mu3.flat)]
    
ax.plot_surface(mu1, mu2, z, cmap=cm.jet, linewidth=0, rstride=1,cstride=1, antialiased=False);
xlabel(r'$\mu_1$')
ylabel(r'$\mu_2$')
ax.set_zlabel(r'$p(\mu)$')

# Gaussians

Central limit theorem:
> "when independent random variables are added, their properly normalized sum tends toward a normal distribution (a bell curve) even if the original variables themselves are not normally distributed."
>
> ~~ yup, that's right. I just quoted [wikipeda](https://en.wikipedia.org/wiki/Central_limit_theorem) (sorry!)

In [ ]:
import scipy.stats

In [ ]:
fig, axes = subplots(1, 3, sharey=True, figsize=(10, 3))

sca(axes[0])
title(r'$x \approx U(x)$')
x = uniform(0, 1., 100000)
hist((x-x.mean())/x.std(), bins=100, normed=True);
plot(linspace(-5,5,100), scipy.stats.norm.pdf(linspace(-5,5,100)))

sca(axes[1])
title('Average of 2')
x += uniform(0,1., 100000)
hist((x-x.mean())/x.std(), bins=100, normed=True);
plot(linspace(-5,5,100), scipy.stats.norm.pdf(linspace(-5,5,100)))

sca(axes[2])

N = 10
title('Average of {}'.format(N))
for k in range(2, N):
    x += uniform(0,1., 100000)
hist((x-x.mean())/x.std(), bins=100, normed=True);

plot(linspace(-5,5,100), scipy.stats.norm.pdf(linspace(-5,5,100)))



A one-dimensional gaussian (bell curve) is 

$$N(x|\mu, \sigma) \propto \exp\left\{-\frac{(x-\mu)^2}{2\sigma^2}\right\}$$

If one integretes the RHS to normalize it the we find that 
$$ N(x|\mu, \sigma) = \frac{1}{\sqrt{2\pi}\sigma}  \exp\left\{-\frac{(x-\mu)^2}{2\sigma^2}\right\}$$

I can confirm this using `sympy` to evaluate the integral

In [ ]:
import sympy
from sympy import oo
sympy.init_printing()

x= sympy.symbols('x')
mu= sympy.symbols('mu', real=True)
sigma = sympy.symbols('sigma', real=True, positive=True)

sympy.integrate(sympy.exp(-(x-mu)**2 /(2*sigma**2)), (x,-oo, oo))

For multivariate $\mathbf{x}=[x_1, x_2, \dots x_K]^T$ we have have the _multivariate_ normal distribution 
$$N(\mathbf{x}|\boldsymbol{\mu}, \boldsymbol{\Sigma}) \propto \exp\left\{-\frac{1}{2} (\mathbf{x}-\boldsymbol{\mu})^T \boldsymbol{\Sigma}^{-1} (\mathbf{x}-\boldsymbol{\mu}) \right\}$$

I am delibrately _not_ normalizing it because I want to make sure we understand the shape of the distribution for now.

Here $\mathbf{\Sigma}$ is a _covariance_ matrix, with 
$$\begin{align}
\mathbf{\Sigma}_{ij} 
&= \mathbb{E}[(x_i-\mu_i)(x_j-\mu_j)]\\
&=\mathbb{E}[x_i x_j] - \mathbb{E}[x_i]\mathbb{E}[x_j]\\
\end{align}
$$
Because $x_ix_j = x_jx_i$, $\mathbf{\Sigma}$ is a symmetric matrix.

To help us understand Gaussian distributions, I think it is helpful to consider the 2D case starting with the _standard_ normal distribution with $\boldsymbol{\mu}=\mathbf{0}$ covariance $\mathbf{I}$, so 
$$\begin{align}
p(\mathbf{x})
&=N(\mathbf{x}|\mathbf{0}, \mathbf{I})\\
&\propto \exp\left\{-\frac{1}{2}\mathbf{x}^T\mathbf{x}\right\}\\
&= \exp\left\{-\frac{||\mathbf{x}||^2}{2}\right\}\\
\end{align}
$$

Here we can see tha the probability depends on the square of the _distance_ of $\mathbf{x}$ from the origin (mean). In this standard multivariate normal distribution the variance is equal (to one) in all directions. I will show contours at 1, 2, 3, and 4 standard deviations ....

In [ ]:
x_1, x_2 = mgrid[-6:6:100j, -6:6:100j]
y = zeros_like(x_1)

y.flat = [scipy.stats.multivariate_normal.pdf(x, cov=eye(2)) for x in zip(x_1.flat, x_2.flat)]

figure()
ax = gca()
imshow(y, extent=(-6,6,-6,6), origin='lower', alpha=0.5, cmap=cm.gray)
clabel(contour(x_1, x_2, y, cmap=cm.jet, levels= [5e-5,0.002,0.022,0.097]))
grid()

This gaussian "poof ball" represents completely uncorrelated "whitened" noise. 